### I'm using this script to test out webscraping using beautifulsoup package.

Basic setup and site access.

In [ ]:
import requests

MainURL='https://www.americanrhetoric.com/top100speechesall.html'
MainPage= requests.get(MainURL, headers={'user-agent': 'Mozilla/5.0'})

Obtaining URL list from main webpage and storing them as list variable.

In [ ]:
from bs4 import BeautifulSoup
soup= BeautifulSoup(MainPage.content, 'html.parser')
mainlinks=[a['href'] for a in soup.find_all('a',href=True)   
    if 'off site' in a.text 
    or a['href'].startswith('speeches') and 'PDFFiles' not in a['href']
    or 'Belief and Public Morality' in a.text]

Test script for first speech, will replace URL with iteration over URL list mainlinks.

In [116]:
URL='https://www.americanrhetoric.com/'+mainlinks[0]
Page=requests.get(URL, headers={'user-agent': 'Mozilla/5.0'})
speechsoup=BeautifulSoup(Page.content, 'html.parser')
for p in speechsoup.find_all("font",{'face':'Verdana'}):
    print(p.get_text(strip=True))

I am happy to join with you today in what will go down in history as
the greatest demonstration for freedom in the history of our nation.
Five score years ago,a great American, in whose symbolic shadow we
stand today, signed theEmancipation Proclamation. This momentous decree came as a great
beacon light of hope to millions of Negro slaves who had been seared in the flames of
withering injustice. It came as a joyous daybreak to end the long night of their
captivity.
But one hundred years later, the Negro still is not free. One
hundred years later, the life of the Negro is still sadly crippled by the manacles of
segregation and the chains of discrimination. One hundred years later, the Negro lives on a
lonely island of poverty in the midst of a vast ocean of material prosperity. One hundred
years later, the Negro is still languished in the corners of American society and finds
himself an exile in his own land. And so we've come here today to dramatize a shameful
condition.
In a sense we